## Problem number 791
Let $S(n)$ be the sum of all quadruples of integers $(a, b, c, d)$ s.t. $ 1 \le a \le b \le c \le d \le n$ such that their average is exactly twice their variance.   
S(5) = 48 (1, 1, 1, 3), (1, 1, 3, 3), (1, 2, 3, 4), (1, 3, 4, 4), (2, 2, 3, 5).  
S(1000) = 37048340.  
Find $S(10^8) \text{ mod } 433494437$ 

We notice that the problem quickly becomes intractable when considering looping through all possible quadruples. So we seek to shrink the input space.   
First we notice that when the first three integers are given, there are exactly two values that make the mean twice the variance, and if these numbers are not an integer or are smaller than the others, we can throw out the quad.   

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt

This means we only have to iterate over a, b, and c. But this is still to large of a search space. Next we try to throw out more combinations if a, b, c does not return a number then a, b, c+1 also does not. 

In [8]:
# 1 <= a <= b <= c <= d <= n
n = 1000
total = 0
count = 0
for d in np.arange(n, 0, -1, dtype=np.float64):
    min_c, max_c = 0, d
    if (sqrt := 24 * d + 9) >= 0: 
        sqrt = math.sqrt(sqrt)
        min_c, max_c = (
            int(((3 * d + 3) - sqrt) / 3) - 1,
            int(((3 * d + 3) + sqrt) / 3) + 1,
        )
    for c in np.arange(min(d, max_c), max(min_c, 0), -1):
        if (
            sqrt := (2 * (1 + c + d)) ** 2
            - 8 * ((3 / 2) * c**2 - d * c + (3 / 2) * d**2 - d - c)
        ) >= 0:
            min_b, max_b = 0, c
            sqrt = math.sqrt(sqrt)
            min_b = int(((2 + 2 * c + 2 * d) - sqrt) / 4 - 0.0001)
        for b in np.arange(c, max(0, min_b), -1):
            # count += 1
            sum_max = d + c + 2 * b
            a = (1 + b + c + d
                - math.sqrt((-1 - b - c - d) ** 2
                + 6 * (b + c + d + b * c + b * d + c * d
                        - (3 / 2) * (b * b + c * c + d * d)))) / 3
            if not int(a) == a:
                continue
            elif 1 <= a <= b:
                total += a + b + c + d
                if total > 433494437:
                    total %= 433494437
 
print(total, count, count / n**2)

37048340.0 0 0.0


Any set of a, b, c, and d must equal zero when plugged into the following polynomial

In [ ]:
def huge_func(a, b, c, d):
    alpha, beta, gamma, delta = a / 4, b / 4, c / 4, d / 4
    return (
        6 * alpha**2
        + (-4) * (alpha * beta + alpha * gamma + alpha * delta)
        - alpha
        + 6 * beta**2
        + (-4) * (beta * gamma + beta * delta)
        - beta
        + 6 * gamma**2
        - 4 * gamma * delta
        - gamma
        + 6 * delta**2
        - delta
    )